<a href="https://colab.research.google.com/github/Untick/resume-builder-gr1/blob/main/Zabigullin%20Vadim/%D0%9F%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%BA%D0%B0_%D0%BF%D1%80%D0%B0%D0%B2%D0%BE%D0%BF%D0%B8%D1%81%D0%B0%D0%BD%D0%B8%D1%8F_%D0%B2_%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обращение к ChatGPT по API. Роли. Влияние инструкции на ответ модели.


Языковые модели, такие как GPT, создаются компанией OpenAI, и с каждой новой версией (от GPT-1 до последней, GPT-4), модель становится всё более мощной и точной. Это означает, что она может создавать более качественные и разнообразные тексты на основе предоставленного вами ввода.

Если вы используете платную подписку **ChatGPT Plus**, у вас есть доступ к моделям GPT-3.5 и GPT-4. Это позволяет вам выбирать модель, которая наилучшим образом подходит для ваших нужд, в зависимости от конкретного применения. Например, GPT-4 может быть более полезен для более сложных задач, требующих более точного понимания и генерации текста.

Через API OpenAI вы можете получить доступ к большему количеству моделей, хотя некоторые из них являются старыми версиями. Доступ к GPT-4 через API предоставляется только после одобрения OpenAI, поскольку это наиболее мощная модель и ее использование может потребовать большего контроля и ресурсов.

Для большинства пользователей API наиболее подходящей и экономичной моделью будет gpt-3.5-turbo. Она предлагает оптимальное сочетание стоимости и производительности, что делает ее хорошим выбором для большинства приложений.

Важно помнить о токенах при использовании этих моделей. Токены - это единицы текста, которые модель обрабатывает при каждом запросе. Количество токенов, используемых в каждом запросе, влияет на стоимость использования модели. Более длинные тексты потребуют больше токенов, и следовательно, будут стоить больше. Поэтому очень важно контролировать использование токенов, особенно при работе с большими объемами данных или при выполнении большого количества запросов. Это особенно важно при использовании GPT-4, поскольку эта модель может быстро исчерпать ваш бюджет. К подсчету токенов мы еще вернемся в данном занятии.

Давайте посмотрим, как обращаться к GPT через API.

Конечно же, для работы с GPT через API вам необходимо получить API ключ. Подробная инструкция, как подключиться к ChatGPT из России и оплатить использование GPT по API находится тут: https://docs.google.com/document/d/1Xnzx8Ve0gncUQREnnGMEa2IukNmYqZp0wD_3PTpWZSM/edit?usp=sharing

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 801.9 kB/s eta 0:00:00


Посмотрим, как можно взаимодействовать с моделью общения OpenAI (в данном случае, "gpt-3.5-turbo") через их API. Передадим сообщение в модель и попросим ее сгенерировать ответ.

ChatGPT использует концепцию "ролей" для различных участников в диалоге.
Каждое сообщение в списке messages является словарем, имеющим два поля: role и content. Role может принимать одно из трех значений: "system", "user" или "assistant", указывая, кто именно "говорит" сообщение. Content содержит само сообщение.

 В стандартном случае, три основные роли могут быть включены в чат:

* **"system"**: Системная роль используется для установки контекста диалога. Это как общая инструкция для модели, которая определяет её поведение во время диалога. Сообщения от "system" обычно помещаются в начале диалога и могут содержать информацию, как модель должна вести себя в роли ассистента.

* **"user"**: Это роль для человека- пользователя, который ведет диалог с моделью. Ваши запросы к модели будут выставлены под этой ролью.

* **"assistant"**: Это роль для модели AI, которая отвечает на запросы от пользователя.

При создании чата с моделью, вы предоставляете список сообщений. Каждое сообщение в этом списке — это словарь, содержащий два элемента: "role" и "content". "Role" указывает на роль (вышеупомянутые "system", "user" или "assistant"), а "content" содержит фактический текст сообщения.

In [2]:
import getpass
import openai
import os
# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key


OpenAI API Key:··········


Для начала, создадим переменные, в которых будут находится тексты, которые мы будем подавать в system и user:

In [10]:
system = '''Ты опытный и старательный специалист-лингвист, который занимается коррекцией ошибок в резюме соискателей работы в области разработки искусственного интеллекта, написанных русским текстом.
У тебя есть инструкция, в которой описано как правильно делать коррекцию русского текста. Тебе предоставят текст, скорректируй его, если это необходимо, опираясь на эту инструкцию.
Корректируй текст максимально точно по инструкции. Не упоминай инструкцию и примеры текста из неё при ответе. Построй грамматически правильно предложение и исправь все ошибки в нём.
Например такой текст: "Каторая на стале калбаса сьела нихарошая лижала сабака ее" после исправления должен быть таким:
"Нехорошая собака съела её колбасу, которая лежала на столе".'''

base_info= '''Понимание задачи: Перед началом работы удостоверьтесь, что вы понимаете контекст и цель текста. Это поможет вам правильно интерпретировать предложения и выявить ошибки.
Проверка орфографии: Внимательно проверьте орфографию слов. Обратите внимание на правильность написания букв, прописных и строчных букв, использование буквы ё и другие орфографические правила.
Проверка пунктуации: Убедитесь, что пунктуация в тексте соответствует правилам. Проверьте запятые, точки, тире, кавычки и другие знаки препинания.
Согласование времен и лиц: Проверьте согласование времен и лиц в предложениях. Убедитесь, что временные формы глаголов и лица согласованы.
Проверка синтаксиса: Проверьте структуру предложений и их логическую связь. Устраните ошибки, связанные с неправильным порядком слов или непонятной структурой предложений.
Проверка согласования: Удостоверьтесь, что существительные, прилагательные и глаголы согласованы в роде, числе и падеже.
Использование словарей и справочников: Пользуйся словарями, доступными тебе, стилистическими справочниками и грамматическими ресурсами для подтверждения правильности исправлений.
Связность и структура: Обратите внимание на связность текста и его общую структуру. Убедитесь, что переходы между абзацами и идеями логичны.
Контроль за повторами: Проверьте текст на повторяющиеся слова, фразы или идеи и устраните их, если это необходимо.
Проверка стиля и тона: Учтите стиль и тональность текста в соответствии с его назначением. В случае необходимости, предложите исправления, чтобы улучшить стиль и язык текста.
Финальное чтение: После всех исправлений выполните финальное чтение текста для проверки, что все ошибки были устранены и текст готов к публикации.'''

#question = '''Я на поле по тракттаре еххал'''
#question = '''утюг включить как подробная тут инструкция'''  # Подробная инструкция, как включить утюг, находится тут
#question = '''програмист я опыттныйй со стажемм роботы больше чем читыре года'''
question = '''Праграмист я опыттныйй со стажемм роботы больше чем читыре года. Маи навыки в програмирование такиее имеються как знаю харашо питон, ява, и си плюс плюс'''

In [6]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.3 MB/s eta 0:00:00


In [7]:
import tiktoken

In [8]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """Возвращает количество токенов, используемых списком сообщений."""
    try:
        encoding = tiktoken.encoding_for_model(model) # Пытаемся получить кодировку для выбранной модели
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base") # если не получается, используем кодировку "cl100k_base"
    if model == "gpt-3.5-turbo-0301" or "gpt-3.5-turbo-0613" or "gpt-3.5-turbo-16k" or "gpt-3.5-turbo":
        num_tokens = 0 # начальное значение счетчика токенов
        for message in messages: # Проходимся по каждому сообщению в списке сообщений
            num_tokens += 4  # каждое сообщение следует за <im_start>{role/name}\n{content}<im_end>\n, что равно 4 токенам
            for key, value in message.items(): # итерация по элементам сообщения (роль, имя, контент)
                num_tokens += len(encoding.encode(value)) # подсчет токенов в каждом элементе
                if key == "name":  # если присутствует имя, роль опускается
                    num_tokens += -1  # роль всегда требуется и всегда занимает 1 токен, так что мы вычитаем его, если имя присутствует
        num_tokens += 2  # каждый ответ начинается с <im_start>assistant, что добавляет еще 2 токена
        return num_tokens # возвращаем общее количество токенов
    else:
      # Если выбранная модель не поддерживается, генерируем исключение
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}. # вызываем ошибку, если функция не реализована для конкретной модели""")


Проверка правописания

In [21]:
# корявый текст
question = '''Праграмист я опыттныйй со стажемм роботы больше чем читыре года. Маи навыки в програмирование такиее имеються как питон, ява, майСКЛ и си плюс плюс'''

In [22]:
# текст скоррекированный ChatGPT
messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": base_info+ f"{question}" }
      #{"role": "user", "content": question}
      ]

print(f"{num_tokens_from_messages(messages, 'gpt-3.5-turbo-0301')} токенов использовано на вопрос")

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0.0,
    max_tokens=1000
    )
print(completion)
message = completion['choices'][0]['message']['content']
print("Мой текст:", question)
print("Текст скоррекированный ChatGPT:", message)

1094 токенов использовано на вопрос
{
  "id": "chatcmpl-7r2oLOTJTiOJufL92hqJN8irU7mii",
  "object": "chat.completion",
  "created": 1692876545,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\u042f \u043e\u043f\u044b\u0442\u043d\u044b\u0439 \u043f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0441\u0442 \u0441 \u043e\u043f\u044b\u0442\u043e\u043c \u0440\u0430\u0431\u043e\u0442\u044b \u0431\u043e\u043b\u0435\u0435 \u0447\u0435\u0442\u044b\u0440\u0435\u0445 \u043b\u0435\u0442. \u0423 \u043c\u0435\u043d\u044f \u0435\u0441\u0442\u044c \u043d\u0430\u0432\u044b\u043a\u0438 \u043f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u044f \u043d\u0430 Python, Java, MySQL \u0438 C++."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1093,
    "completion_tokens": 43,
    "total_tokens": 1136
  }
}
Мой текст: Праграмист я опыттныйй со 